In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from emonet import EmoNet,Modified_EmoNet
import os
from sklearn.model_selection import train_test_split
import random
random.seed(42)

In [3]:
print(os.path)

<module 'ntpath' from 'c:\\Users\\nalin\\AppData\\Local\\Programs\\Python\\Python310\\lib\\ntpath.py'>


In [5]:
#loading model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmoNet(n_expression=8)
model.load_state_dict(torch.load("D:\Integrated_gap_gradients\ig2_CNN\emonet\pretrained\emonet_8.pth"))  # Adjust path if needed
model = model.to(device)
model.eval()  # Set to evaluation mode

print(model)

EmoNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): ConvBlock(
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (downsample): Sequential(
      (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
  )
  (conv3): ConvBlock(
  

In [3]:
#function to extract relevant emotion logits, we only need 4 instead of 8
def extract_relevant_logits(output):
    logits=output["expression"]
    relevant_logits = torch.stack([
        logits[:, 0],  # N
        logits[:, 6],  # A
        logits[:, 4],  # F
        logits[:, 1],  # (HappyClosedMouth + HappyOpenMouth)
    ], dim=1)  # Shape: (batch_size, 4)

    return relevant_logits

In [6]:
#freezing convolutional layers of the model, unfreeze final layers
#defining optimizer and loss function for training loop
for param in model.parameters():
    param.requires_grad = False

for param in model.emo_fc_2.parameters():
    param.requires_grad = True

optimizer = torch.optim.SGD(model.emo_fc_2.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

In [7]:
#Preparing CFD dataset for training
def create_dataset_dict(source_folder):
    dataset = {}
    emotion_map = {'N': 0, 'A': 1, 'F': 2, 'HC': 3, 'HO': 3}

    for folder in os.listdir(source_folder):
        folder_path = os.path.join(source_folder, folder)
        if not os.path.isdir(folder_path):
            continue
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.jpg'):
                parts = file_name.split('-')
                if len(parts) < 5:
                    continue 
                emotion_key = parts[4].split(".")[0] 
                label = emotion_map.get(emotion_key, None)
                if label is not None:
                    dataset[folder_path+"\\"+file_name] = label
    return dataset

source_directory_path = "D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\CFD"
dataset_dict = create_dataset_dict(source_directory_path)

print(f"Total Images: {len(dataset_dict)}")

Total Images: 1207


In [8]:
dataset_dict

{'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-200\\CFD-AF-200-228-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-201\\CFD-AF-201-060-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-202\\CFD-AF-202-122-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-203\\CFD-AF-203-077-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-204\\CFD-AF-204-067-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-205\\CFD-AF-205-155-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-206\\CFD-AF-206-079-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-207\\CFD-AF-207-023-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-208\\CFD-AF-208-003-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-209\\CFD-AF-209-006-N.jpg': 0,
 'D:\\Integrated_gap_gradients\\ig2_CNN\\gpu_env_ig2\\CFD\\AF-210\\CFD

In [9]:
#creating train-test datasets
from sklearn.model_selection import train_test_split

data_image_paths = list(dataset_dict.keys())
labels = list(dataset_dict.values())
train_files, test_files, train_labels, test_labels = train_test_split(
    data_image_paths, labels, test_size=0.3, random_state=42, stratify=labels
)
train_dataset_dict = dict(zip(train_files, train_labels))
test_dataset_dict = dict(zip(test_files, test_labels))

print(f"Training set size: {len(train_dataset_dict)}")
print(f"Test set size: {len(test_dataset_dict)}")


Training set size: 844
Test set size: 363


In [10]:
#creating Dataset class
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CFD(Dataset):
    def __init__(self,dataset_dict,transform=None):
        self.dataset_dict=dataset_dict
        self.transform=transform
    
    def __len__(self):
        return len(self.dataset_dict)
    
    def __getitem__(self, index):
        file_path=list(self.dataset_dict.keys())[index]
        label=self.dataset_dict[file_path]

        image=Image.open(file_path).convert("RGB")

        if self.transform:
            image=self.transform(image)
        
        return image, torch.tensor(label,dtype=torch.long)  #needs long format for loss computation

In [11]:
#defining image transform
cfd_transform=transforms.Compose([ transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])])

In [12]:
train_dataset=CFD(train_dataset_dict,cfd_transform)
test_dataset=CFD(test_dataset_dict,cfd_transform)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)
sample_images, sample_labels = next(iter(train_loader))
print(f"Batch Shape: {sample_images.shape}, Labels: {sample_labels}")

Batch Shape: torch.Size([32, 3, 256, 256]), Labels: tensor([0, 3, 1, 0, 3, 0, 0, 1, 3, 0, 3, 0, 3, 3, 2, 0, 1, 1, 0, 0, 1, 0, 2, 3,
        0, 0, 2, 0, 0, 0, 3, 0])


In [71]:
for param in model.parameters():
    param.requires_grad = False

for param in model.emo_fc_2.parameters():
    param.requires_grad = True

optimizer = torch.optim.SGD(model.emo_fc_2.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

In [72]:
#TRAINING LOOP

import time

start_time = time.time()
num_epochs = 100
training_dict={}
training_dict["epochs"]=num_epochs
loss_list=[]
accuracy_list=[]

for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    correct, total = 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() 
        outputs = model(images)
        selected_logits = extract_relevant_logits(outputs)
        loss = criterion(selected_logits, labels)
        loss.backward() 
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(selected_logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss_list.append(loss)
        accuracy_list.append(round(100 * (correct / total), 2))

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

end_time = time.time()
elapsed_time = end_time - start_time
training_dict["elapsed_time"]=elapsed_time
training_dict["loss"]=loss_list
training_dict["accuracy"]=accuracy_list

print(f"Execution Time: {elapsed_time:.4f} seconds")
torch.save(model.state_dict(), "finetuned_emonet_100_epochs.pth")
print("Model saved!")


Epoch [1/100], Loss: 1.5126, Accuracy: 79.38%
Epoch [2/100], Loss: 1.4086, Accuracy: 79.86%
Epoch [3/100], Loss: 1.3638, Accuracy: 80.92%
Epoch [4/100], Loss: 1.1683, Accuracy: 79.98%
Epoch [5/100], Loss: 1.1165, Accuracy: 80.69%
Epoch [6/100], Loss: 1.2173, Accuracy: 81.16%
Epoch [7/100], Loss: 1.1207, Accuracy: 82.82%
Epoch [8/100], Loss: 1.0423, Accuracy: 82.58%
Epoch [9/100], Loss: 1.1278, Accuracy: 82.94%
Epoch [10/100], Loss: 0.9962, Accuracy: 84.36%
Epoch [11/100], Loss: 0.9756, Accuracy: 85.07%
Epoch [12/100], Loss: 0.9339, Accuracy: 84.72%
Epoch [13/100], Loss: 0.9114, Accuracy: 84.83%
Epoch [14/100], Loss: 0.9356, Accuracy: 84.72%
Epoch [15/100], Loss: 0.7928, Accuracy: 86.26%
Epoch [16/100], Loss: 0.8092, Accuracy: 86.61%
Epoch [17/100], Loss: 0.7257, Accuracy: 86.26%
Epoch [18/100], Loss: 0.8405, Accuracy: 86.14%
Epoch [19/100], Loss: 0.8078, Accuracy: 85.31%
Epoch [20/100], Loss: 0.7890, Accuracy: 87.09%
Epoch [21/100], Loss: 0.7183, Accuracy: 86.97%
Epoch [22/100], Loss: 

### Model Evaluation

In [11]:
#we load the 100 epoch model
model = EmoNet(n_expression=8)
model.load_state_dict(torch.load("D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\\finetuned_emonet_100_epochs.pth"))
model.to(device)
model.eval()

In [15]:
#function for obtaining output class from emonet outputs for each image
def predict_emonet(image_path, model):
    image = Image.open(image_path).convert("RGB")
    image = cfd_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        predicted_label = torch.argmax(extract_relevant_logits(output), dim=1).item() 
    
    return predicted_label

In [16]:
correct = 0
total = len(test_dataset_dict)

for img_path, true_label in test_dataset_dict.items():
    pred_label = predict_emonet(img_path, model)
    if pred_label == true_label:
        correct += 1

accuracy = correct / total * 100
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 91.74%


### finetuning model_2 post suppression

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_2 = Modified_EmoNet(n_expression=8)
model_2.load_state_dict(torch.load("D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\\finetuned_emonet_100_epochs.pth"))
model_2.to(device)

Modified_EmoNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): ConvBlock(
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (downsample): Sequential(
      (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
  )
  (conv3): Conv

In [21]:
for param in model.parameters():
    param.requires_grad = False

for param in model.emo_fc_2.parameters():
    param.requires_grad = True

optimizer = torch.optim.SGD(model.emo_fc_2.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

In [ ]:
import time

start_time = time.time()
num_epochs = 50
training_dict_2={}
training_dict_2["epochs"]=num_epochs
loss_list_2=[]
accuracy_list_2=[]

for epoch in range(num_epochs):
    model_2.train() 
    running_loss_2 = 0.0
    correct, total = 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() 
        outputs = model_2(images)
        selected_logits = extract_relevant_logits(outputs)
        loss = criterion(selected_logits, labels)
        loss.backward() 
        optimizer.step()
        
        running_loss_2 += loss.item()
        _, predicted = torch.max(selected_logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss_list_2.append(loss)
        accuracy_list_2.append(round(100 * (correct / total), 2))

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss_2/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

end_time = time.time()
elapsed_time = end_time - start_time
training_dict_2["elapsed_time"]=elapsed_time
training_dict_2["loss"]=loss_list_2
training_dict_2["accuracy"]=accuracy_list_2

print(f"Execution Time: {elapsed_time:.4f} seconds")
torch.save(model_2.state_dict(), "finetuned_emonet_100_epochs_postsupp.pth")
print("Model saved!")

KeyboardInterrupt: 